In [ ]:
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client
!pip install -q -U google-generativeai
!pip install --upgrade google-generativeai

In [ ]:
#Import necessary libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai
import time
from google.generativeai.types import RequestOptions
from google.api_core import retry

In [ ]:
# Import the API Key (blinded)
GOOGLE_API_KEY=""
genai.configure(api_key=GOOGLE_API_KEY)

# Reading G-Drive Sheet

In [ ]:
!pip install --quiet gspread oauth2client google-auth
import gspread
from google.colab import auth
import pandas as pd
import google.auth

auth.authenticate_user()

# Authorize gspread client
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

def load_sheet(sheet_id, sheet_name="Sheet1"):
    # Load google sheet as dataframe
    worksheet = gc.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

#(path blinded)
path = ""
sheet = load_sheet(path,"Reacting to Errors Predict_responses")

Get all the responses and scores

In [ ]:
INPUT_COLUMN = "response" # Specify the column from which input is read
row_count = len(sheet) # Get number of rows
inputs = [sheet.loc[i, INPUT_COLUMN] for i in range(row_count)]
inputs = [x for x in inputs if x is not None]
print(inputs)

# Gemini Prompt Setup

Scoring Prompt Input

In [ ]:
GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please score the following tutor response to a tutor training scenario involving a middle school student making a math error as follows:
-If the tutor’s response provides an implicit way of calling attention to the error and focuses on encouraging students thinking about math and  metacognition rather than giving explicit answers, score with a 1. Examples of responses scoring a 1 are: “Lucy, great start! Explain how you passed to the second column”; “Thank you for showing me your work, Lucy. You worked hard on adding these numbers. I am going to work on this problem with you. Can you tell me how you added the numbers first?”; “Well done setting up the problem, Lucy. Can you tell me how you went about calculating the first few steps?”; “I like your effort, but show me what you are trying to do in the second step”; “Kanye, very well done. Your effort was very valuable. Can you repeat how you arrived at this result? Let's do this problem together”; “Hi Aaron, you made a good try. Could you explain your calculation step for me?”; and “I got a different answer. Let's look at this together.”
-If the tutor's response addresses the student’s error using explicit language by directly calling attention to the student’s error, or fails to encourage the student to think about math and reflect on their thought process, score with a 0. Sample responses scoring a 0 include: “Let's try solving the problem together.”; “Good effort.”; ; “Do you know about carrying the 1?”; “Kanye, let me show you how this should be done”; “Great attempt! You've just to change one thing to make the answer correct”; and “You’re very close, but not completely correct. Would you mind walking me through how you set up this problem?”
Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## gemini API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 300
TEMPERATURE = 0
RUN_UP_TO = 256  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
SCORE_COLUMN = "response"  # Change column numbers here to  modify where output is written

MODEL = 'gemini-2.5-pro'
model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)

if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
tempScoreList = []
tempDirectionList = []
tempRationaleList = []
retries = []

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  if index%60 == 0:
    time.sleep(30)
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: " # Gemini Change
  generation_config = genai.GenerationConfig(temperature=TEMPERATURE)
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config,request_options=RequestOptions(retry=retry.Retry(initial=1.0,multiplier=2.0,maximum=60.0,timeout=300.0)))
  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-4]
  # We now need to parse the JSON into rationale and score
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale
    sheet.at[index,"Gemini Score"] = score  # Now write both into the dataframe
    sheet.at[index,"Gemini Rationale"] = rationale
    tempScoreList.append(score)
    tempDirectionList.append(generation_prompt)
    tempRationaleList.append(rationale)

  except:
    print("error!")
    tempScoreList.append("error during LLM evaluation")
    tempRationaleList.append("error during LLM evaluation")
    retries.append((index,inpt))

print(tempScoreList)
print(tempDirectionList)
print(tempRationaleList)

In [ ]:
for i in range(len(inputs_upto)):
  print(inputs_upto[i])
  print(tempScoreList[i])
  print(tempRationaleList[i])
  print()

In [ ]:
print(list(sheet["Gemini Score"]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save Sheet

In [ ]:
#path blinded
sheet.to_csv('', index=False)